# Week 2: Implementing Callbacks in TensorFlow using the MNIST Dataset
Dans ce cours, vous avez appris à effectuer une classification à l'aide de Fashion MNIST, un ensemble de données contenant des vêtements. Il existe un autre jeu de données similaire, appelé MNIST, qui contient des éléments d'écriture manuscrite - les chiffres de 0 à 9.

Ecrivez un classificateur MNIST qui s'entraîne à 99% de précision et s'arrête une fois ce seuil atteint. Dans le cours, vous avez vu comment cela était fait pour le loss, mais ici vous utiliserez accuracy à la place.

Quelques notes :

1. Votre réseau doit réussir en moins de 9 époques (epochs).
2. Lorsqu'il atteint 99% ou plus, il doit imprimer la chaîne "Reached 99% accuracy so cancelling training !" et arrêter l'entraînement.
3. Si vous ajoutez des variables supplémentaires, assurez-vous d'utiliser les mêmes noms que ceux utilisés dans la classe. Ceci est important pour les signatures de fonction (les paramètres et les noms) des callbacks.

In [1]:
import os
import tensorflow as tf
from tensorflow import keras

## Load and inspect the data

Commencez par charger les données. Deux ou trois choses à noter :

- Le fichier `mnist.npz` est déjà inclus dans l'espace de travail actuel sous le dossier `data` . Par défaut, le `load_data` de Keras accepte un chemin relatif à `~/.keras/datasets` mais dans ce cas, il est stocké ailleurs, ce qui fait que vous devez spécifier le chemin complet.

- `load_data` renvoie les ensembles de formation et de test sous la forme de tuples `(x_train, y_train), (x_test, y_test)` mais dans cet exercice, vous n'aurez besoin que de la rame, vous pouvez donc ignorer le deuxième tuple.

In [2]:
# Load the data

# Get current working directory
current_dir = os.getcwd()

# Append data/mnist.npz to the previous path to get the full path
data_path = os.path.join(current_dir, "data/mnist.npz")

# Discard test set
(x_train, y_train), _ = tf.keras.datasets.mnist.load_data(path=data_path)
        
# Normalize pixel values
x_train = x_train / 255.0

Maintenant, regardez la forme des données de formation :

In [3]:
data_shape = x_train.shape

print(f"There are {data_shape[0]} examples with shape ({data_shape[1]}, {data_shape[2]})")

There are 60000 examples with shape (28, 28)


## Defining your callback

Il est maintenant temps de créer votre propre callback. Pour cela, il faut compléter le `myCallback` class et le `on_epoch_end` method dans la cellule ci-dessous. Si vous avez besoin de conseils sur la façon de procéder, consultez le site suivant [link](https://www.tensorflow.org/guide/keras/custom_callback).

In [4]:
# GRADED CLASS: myCallback
### START CODE HERE

# Remember to inherit from the correct class
class myCallback(keras.callbacks.Callback):
        # Define the correct function signature for on_epoch_end
        def on_epoch_end(self, epoch, logs={}):
            if logs.get('accuracy') is not None and logs.get('accuracy') > 0.99:
                print("\nReached 99% accuracy so cancelling training!") 
                
                # Stop training once the above condition is met
                self.model.stop_training = True

### END CODE HERE


## Create and train your model

Maintenant que vous avez défini votre callback, il est temps de compléter le processus d'intégration. `train_mnist` fonction ci-dessous. 

**Vous devez configurer votre modèle pour qu'il s'entraîne pendant 10 époques et la fonction de rappel doit se déclencher avant la 9e époque pour que vous réussissiez cette tâche.**

**Indice:**
- N'hésitez pas à essayer l'architecture du réseau neuronal qui vous convient, mais si vous avez besoin d'une aide supplémentaire, vous pouvez consulter une architecture qui fonctionne plutôt bien à la fin de ce notebook.

In [8]:
# GRADED FUNCTION: train_mnist
def train_mnist(x_train, y_train):

    ### START CODE HERE
    
    # Instantiate the callback class
    callbacks = myCallback()
    
    # Define the model
    model = tf.keras.models.Sequential([ 
        keras.layers.Flatten(),
        keras.layers.Dense(512, activation=tf.nn.relu),
        keras.layers.Dense(10, activation=tf.nn.softmax)
    ]) 

    # Compile the model
    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy']) 
    
    # Fit the model for 10 epochs adding the callbacks
    # and save the training history
    history = model.fit(x_train, y_train, epochs=10, callbacks=[callbacks])

    ### END CODE HERE

    return history

Call the `train_mnist` passing in the appropiate parameters to get the training history:

In [9]:
hist = train_mnist(x_train, y_train)

Epoch 1/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.1999 - accuracy: 0.9413
Epoch 2/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0805 - accuracy: 0.9751
Epoch 3/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0506 - accuracy: 0.9838
Epoch 4/10
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0353 - accuracy: 0.9892
Epoch 5/10
1870/1875 [============================>.] - ETA: 0s - loss: 0.0280 - accuracy: 0.9907
Reached 99% accuracy so cancelling training!
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0280 - accuracy: 0.9908


Si vous voyez le message `Reached 99% accuracy so cancelling training!` s'affiche après moins de 9 époques, cela signifie que votre callback a fonctionné comme prévu. 

## Need more help?

Exécutez la cellule suivante pour voir une architecture qui fonctionne bien pour le problème en question :

In [10]:
# WE STRONGLY RECOMMEND YOU TO TRY YOUR OWN ARCHITECTURES FIRST
# AND ONLY RUN THIS CELL IF YOU WISH TO SEE AN ANSWER

import base64

encoded_answer = "CiAgIC0gQSBGbGF0dGVuIGxheWVyIHRoYXQgcmVjZWl2ZXMgaW5wdXRzIHdpdGggdGhlIHNhbWUgc2hhcGUgYXMgdGhlIGltYWdlcwogICAtIEEgRGVuc2UgbGF5ZXIgd2l0aCA1MTIgdW5pdHMgYW5kIFJlTFUgYWN0aXZhdGlvbiBmdW5jdGlvbgogICAtIEEgRGVuc2UgbGF5ZXIgd2l0aCAxMCB1bml0cyBhbmQgc29mdG1heCBhY3RpdmF0aW9uIGZ1bmN0aW9uCg=="
encoded_answer = encoded_answer.encode('ascii')
answer = base64.b64decode(encoded_answer)
answer = answer.decode('ascii')

print(answer)


   - A Flatten layer that receives inputs with the same shape as the images
   - A Dense layer with 512 units and ReLU activation function
   - A Dense layer with 10 units and softmax activation function



**Congratulations on finishing this week's assignment!**

You have successfully implemented a callback that gives you more control over the training loop for your model. Nice job!

**Keep it up!**